# Домашнее задание по теме "Продвинутый pandas"

## Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [2]:
import pandas as pd

def get_source_type(row):
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        return 'organic'
    elif row['traffic_source'] == 'paid' or row['traffic_source'] == 'email':
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return row['traffic_source']

visits = pd.read_csv('visit_log.csv', sep = ';')
visits['source_type'] = visits.apply(get_source_type, axis=1)
visits.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


## Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафреймом<br>
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [10]:
urls = pd.read_csv('URLs.txt')
urls.loc[urls.url.str.contains(r'\/\d{8}-', regex=True)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [59]:
from datetime import datetime
from datetime import timedelta

In [52]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [107]:
# отфильтруем пользователей с числом оценок больше 100
filtered = ratings.groupby('userId').filter(lambda x: len(x) > 100)
filtered

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [99]:
# для каждого пользователя посчитаем время жизни
lifetimes = filtered.groupby('userId').apply(lambda row: row.timestamp.max() - row.timestamp.min())
lifetimes

userId
1         739163
4       62496114
6           3806
7       69546315
10        317722
         ...    
605        82248
606    197231731
607     34768995
608     72402187
610     16417448
Length: 245, dtype: int64

In [105]:
# посчитаем среднее время жизни и приведём к читабельному формату
dt = datetime.fromtimestamp(lifetimes.mean())
# преобразуем результат в количество дней и часов как разницу между полученным timestamp и началом эпохи
dt_zero = datetime.fromtimestamp(0)
delta = dt - dt_zero
print(f'Среднее время жизни: {delta.days} дней, {round(delta.seconds / 3600)} часов')

Среднее время жизни: 517 дней, 17 часов


## Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [4]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto


,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [5]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [14]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
joined = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id').fillna(0)
joined

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [16]:
joined_address = joined.merge(client_base, on='client_id')
joined_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
